In [32]:
import pandas as pd
from pandas.core.common import flatten
import numpy as np
import random
np.set_printoptions(suppress=True)



Skipping line 3135037: unexpected end of data
Skipping line 3135066: unexpected end of data
Skipping line 3135074: unexpected end of data


### Loading the data

In [40]:
data = pd.read_csv ('../Experiments/model experiment-table2.csv', skiprows=6, nrows=3135000, engine='python', error_bad_lines=False)
#get data for different agents
security = data.iloc[:, 16:]
criminal = data.iloc[:, 5:16]
meta = data.iloc[:,:5]

### convert the memory from a string to array of strings 

In [41]:
#converts memory from one string to an array of strings for each item in memory. Then sort according to last seen 
def sort_memory(row):
    new_row = row.split('] [')
    new_row[0] = new_row[0][2:]
    new_row[-1] = new_row[-1][:-2]
    n_values = len(new_row)
    new_row = np.array(new_row)
    new_row = new_row[::-1]
    return(new_row)

#create new array for memory
memory_security = security['memory-security']
memory_criminal = criminal['memory-criminal']

memory_security = memory_security.apply(sort_memory)
memory_criminal = memory_criminal.apply(sort_memory)

### Pad the memory 

Since we want to have a memory of uniform size, no matter how many people a person has seen we need to pad those with smaller memories 

In [42]:

#pad memory
def padded_memory(memory, memory_size, printing=False):
    
    length = memory.shape[0]
    padding = np.zeros((length,memory_size*6))
    indexes_for_mem =  []
    
    for row in range(length):
        if len(memory[row]) > memory_size:
            memory[row] = memory[row][:memory_size] # if we make the memory smaller than what we have captured
            
        memory_items = []
        
        if memory[row][0] != '': # here sample indicies to put memories in random order 
            memory_indexes = random.sample(range(0, padding.shape[1]//6), len(memory[row]))
        else:
            memory_indexes = []
            
        indexes_for_mem.append(memory_indexes)
        
        for index, memory_index in enumerate(memory_indexes):
           
            value = memory[row][index]  #get array, and split it into numbers
            split = value.strip('[')
            split = split.split(' ')
            items_needed = []
            
            #select items and deal with nobodys if they exist
            if split[0] != "nobody":
                items_needed.append(split[2:5])
                items_needed.append(split[9:])
            elif split[0] == "nobody":
                items_needed.append(split[1:4])
                items_needed.append(split[8:])
                
            #put them into an array for all items in that row of the memory
            items_needed = np.array(list(flatten(items_needed)),dtype=float)
            for i in range(len(items_needed)):
                padding[row, memory_index*6 + i] = items_needed[i]
    
    return(padding, indexes_for_mem)

In [43]:
memory_size = 20

padded_criminal_memory, crim_indexes = padded_memory(memory_criminal,memory_size)
padded_security_memory, sec_indexes = padded_memory(memory_security,memory_size)

In [44]:
#creates input data with the padded memory items and other items pulled from data
other_items_criminal = criminal[["current-x-cor-criminal","current-y-cor-criminal","criminal-cash","current-platform-criminal"]].values
other_items_security = security[["current-x-cor-security","current-y-cor-security","current-platform-security"]].values

n_data_points_crim  = padded_criminal_memory.shape[0]
n_data_points_sec = padded_security_memory.shape[0]

n_additional_inputs_crim = other_items_criminal.shape[1]
n_additional_inputs_sec = other_items_security.shape[1]


input_data_crim = np.zeros((n_data_points_crim,memory_size*6+n_additional_inputs_crim))
input_data_sec = np.zeros((n_data_points_sec,memory_size*6+n_additional_inputs_sec))

input_data_crim[:,:memory_size*6] = padded_criminal_memory
input_data_crim[:,memory_size*6:] = other_items_criminal

input_data_sec[:,:memory_size*6] = padded_security_memory
input_data_sec[:,memory_size*6:] = other_items_security

In [45]:
target_security = security[['target-security']].values
objective_security = security[['objective-security']].values

target_criminal = criminal[['target-criminal']].values
objective_criminal = criminal[['objective-criminal']].values

#doesn't take the padded memory as an input, takes the sort_memory output 
def get_one_hot(target,objective,memory,memory_size, mem_index):
    n_data_points = memory.shape[0]
    
    #create an array for one-hot indices
    one_hot_indices = np.zeros(n_data_points)
    
    #for each in objective insert according index to indices 
    #try-except deals with a value error when the target can't be found as an item in the memory
    for i in range(n_data_points):

        mem_i = mem_index[i]
        
        if objective[i] == 'investigate':
            
            index = np.array(np.where(memory[i] == target[i][0][1:-1]))[0]
            if len(index) > 0:
                index = index[0]
 
            try:
                one_hot_indices[i] = mem_i[index]
            except :
                one_hot_indices[i] = memory_size*2
        if objective[i] == 'arrest_target' or objective[i] == 'steal_from_target':
            index = np.array(np.where(memory[i] == target[i][0][1:-1]))[0]
            if len(index) > 0:
                index = index[0]
  
            try:
                one_hot_indices[i]= memory_size +  mem_i[index]
            except :
                one_hot_indices[i] = memory_size*2
        if objective[i] == 'explore':
            one_hot_indices[i] = memory_size*2
        if objective[i] == 'leave':
            one_hot_indices[i] = memory_size*2 + 1
        

    
    return(one_hot_indices)

one_hot_sec = get_one_hot(target_security,objective_security,memory_security,memory_size, sec_indexes)
one_hot_crim = get_one_hot(target_criminal,objective_criminal,memory_criminal,memory_size, crim_indexes)
        

In [47]:
#save to file

np.savetxt("../Data/target_sec_large.csv", one_hot_sec, delimiter=",")
np.savetxt("../Data/target_crim_large.csv", one_hot_crim, delimiter=",")
np.savetxt("../Data/input_data_sec_large.csv", input_data_sec, delimiter=",")
np.savetxt("../Data/input_data_crim_large.csv", input_data_crim, delimiter=",")